Import Dependencies

In [33]:
import subprocess, os, json

In [12]:
libraries =["folium", "meteomatics", "matplotlib", "pytz", "requests"]

def install_and_import_libraries(list_of_libraries):
    for library in list_of_libraries:
        try:
            # Installer la librairie
            result = subprocess.run(["pip", "install", library, "--user"], capture_output=True, text=True, check=True)
            for line in result.stdout.splitlines():
                if not line.startswith("Requirement already satisfied"):
                    print(line)
            # Importer la librairie après installation
            globals()[library] = __import__(library)
        except Exception as e:
            print(f"Error installing or importing {library}: {e}")


install_and_import_libraries(libraries)

import time
import requests, pandas as pd
from datetime import datetime, timedelta, timezone, time
import os
from pathlib import Path
import meteomatics.api as api
import matplotlib.pyplot as plt
import pytz

Présentation Location de la Bouée 51101

In [38]:
# Coordonnées du point
latitude = 24.359
longitude = -162.081

# Making a variable of it for the API call later in the code
coordinates = [(latitude, longitude)]

# Création de la carte centrée sur le point
buoy_location_map = folium.Map(location=[latitude, longitude], zoom_start=3)

# Ajouter un marqueur pour la bouée
folium.Marker([latitude, longitude], popup="Bouée NDBC").add_to(buoy_location_map)

# Afficher la carte
buoy_location_map

Get Credentials

In [39]:
# Chemin du fichier JSON
file_path = r"C:\\Users\\f.gionnane\\Downloads\\Projects\\credentials.json"

# Lecture du fichier JSON
def read_credentials(file_path):
    with open(file_path, 'r') as file:
        credentials = json.load(file)
        
        print(f"Utilisateur : {credentials['username']}")
        print(f"Mot de passe : {credentials['password']}")
        meteo_username = credentials['username']
        meteo_password = credentials['password']
        
    return meteo_username, meteo_password

read_credentials(file_path)

Utilisateur : self_gionnane_florian
Mot de passe : mQCnw260I1


('self_gionnane_florian', 'mQCnw260I1')

Gather Last 24h Climate Data for Hawaii 51101 Station Buoy Zone by 10 minutes intervals

In [43]:
# Définir les paramètres
parameters = [
        'wind_speed_10m:ms',	
        'wind_dir_10m:d',
        'wind_gusts_10m_1h:ms',
        't_2m:C',
        't_max_2m_24h:C',
        't_min_2m_24h:C',
        'msl_pressure:hPa',
        'precip_1h:mm'
    ]
# Définir les coordonnées
coordinates = [(24.359, -162.081)]

# Définir le modèle
model = 'mix'
# Début URL de l'API
DEFAULT_API_BASE_URL = "https://api.meteomatics.com"
enddate = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0)
startdate = enddate - timedelta(days=1)
interval = timedelta(minutes=10)

# Fonction de requête API Météo
def meteo_api_request(parameters, coordinates, model, DEFAULT_API_BASE_URL, startdate, enddate, interval):

    # Appel de la fonction read_credentials pour obtenir les informations d'identification
    meteo_username, meteo_password = read_credentials(file_path)
    
    # Récupérer les données
    meteo_df = api.query_time_series(coordinates, startdate, enddate, interval, parameters, meteo_username, meteo_password, model=model)
    meteo_df.reset_index(inplace=True)
    return meteo_df

meteo_df=meteo_api_request(parameters, coordinates, model, DEFAULT_API_BASE_URL, startdate, enddate, interval)
meteo_df.head() 

Utilisateur : self_gionnane_florian
Mot de passe : mQCnw260I1


,lat,lon,validdate,wind_speed_10m:ms,wind_dir_10m:d,wind_gusts_10m_1h:ms,t_2m:C,t_max_2m_24h:C,t_min_2m_24h:C,msl_pressure:hPa,precip_1h:mm
0,24.359,-162.081,2025-02-02 19:00:00+00:00,2.0,354.6,5.7,21.4,21.5,20.9,1018.9,0.0
1,24.359,-162.081,2025-02-02 19:10:00+00:00,3.0,353.5,5.3,21.4,21.5,20.9,1019.0,0.0
2,24.359,-162.081,2025-02-02 19:20:00+00:00,3.0,352.3,5.0,21.5,21.5,20.9,1019.1,0.0
3,24.359,-162.081,2025-02-02 19:30:00+00:00,3.0,351.1,4.6,21.4,21.5,20.9,1019.1,0.0
4,24.359,-162.081,2025-02-02 19:40:00+00:00,2.0,349.8,4.3,21.6,21.6,20.9,1019.2,0.0


In [5]:
meteo_df.rename(columns={'validdate': 'Datetime', 'wind_speed_10m:ms':'wind speed (ms)','wind_dir_10m:d':'wind dir (°)',
                          'wind_gusts_10m_1h:ms':'wind_gusts 1h (ms)','t_2m:C':'T° (C°)','t_max_2m_24h:C':'T°_max 24h (C°)',
                          't_min_2m_24h:C':'T° min 24h (°)', 'msl_pressure:hPa': 'sea level pressure (hPa)','precip_1h:mm':'Precip 1h (mm)'}, inplace=True)
meteo_df = meteo_df.drop(columns=['lat','lon'])
meteo_df.head()

,Datetime,wind speed (ms),wind dir (°),wind_gusts 1h (ms),T° (C°),T°_max 24h (C°),T° min 24h (°),sea level pressure (hPa),Precip 1h (mm)
0,2024-05-20 14:00:00+00:00,5.0,161.7,5.0,24.0,24.5,21.8,1015.8,0.81
1,2024-05-20 14:10:00+00:00,5.0,159.7,5.0,24.0,24.5,21.8,1015.9,0.63
2,2024-05-20 14:20:00+00:00,5.0,157.3,5.0,24.0,24.5,21.8,1015.9,0.63
3,2024-05-20 14:30:00+00:00,6.0,154.4,6.0,24.1,24.5,21.8,1015.9,0.63
4,2024-05-20 14:40:00+00:00,6.0,151.0,6.0,23.9,24.5,21.8,1016.0,0.63


In [6]:
meteo_df.dtypes

Datetime                    datetime64[ns, UTC]
wind speed (ms)                         float64
wind dir (°)                            float64
wind_gusts 1h (ms)                      float64
T° (C°)                                 float64
T°_max 24h (C°)                         float64
T° min 24h (°)                          float64
sea level pressure (hPa)                float64
Precip 1h (mm)                          float64
dtype: object

Get Google Big Query Credentials And Send the Data

In [7]:
meteo_df.columns

Index(['Datetime', 'wind speed (ms)', 'wind dir (°)', 'wind_gusts 1h (ms)',
       'T° (C°)', 'T°_max 24h (C°)', 'T° min 24h (°)',
       'sea level pressure (hPa)', 'Precip 1h (mm)'],
      dtype='object')

In [8]:
meteo_df_final= meteo_df
meteo_df_final.to_csv('df_meteo_final.csv', index=False)